Import Libraries

In [ ]:
import os
import sys
from getpass import getpass
import torch.optim as optim
!pip install dropbox > /dev/null

Configure Environment

In [ ]:
os.environ["ENVIRONMENT"] = "colab"
os.environ["REPO_DIR"] = "/content/UMARV-CV-LaneDetection"
os.environ["ROOT_DIR"] = "/content"
# Insert MODEL_ID here
os.environ["MODEL_ID"] = ""
os.environ["MODEL_DIR"] = f"{os.getenv('REPO_DIR')}/models/model_{os.getenv('MODEL_ID')}"

Configure git

In [ ]:
# Fill in branch
git_branch = ""

while not git_branch:
    git_branch = input("Enter your branch: ")

git_repo_url = "https://github.com/AwrodHaghiTabrizi/UMARV-CV-LaneDetection.git"
!git clone -b $git_branch $git_repo_url
%cd "{os.getenv('REPO_DIR')}"

Import Repository Resources

In [ ]:
sys.path.insert(0, f"{os.getenv('REPO_DIR')}/src")
from helpers import *

sys.path.insert(0, f"{os.getenv('MODEL_DIR')}/src")
from methods import *
from architecture import *
from dataset import *

Download Datasets

In [ ]:
dbx_access_token = getpass("Enter your DropBox access token: ")

In [ ]:
# download_datasets_from_dropbox(
#     dbx_access_token = dbx_access_token,
#     include_all_datasets = False,
#     use_thread = True
# )

# upload_datasets_to_google_drive()

In [ ]:
# get_datasets_from_google_drive()

Code

In [5]:
num_epochs = 15
batch_size = 32
val_size = 50

In [ ]:
device = set_device()
model = initialize_model(device=device, dbx_access_token=dbx_access_token)

train_dataset, val_dataset, benchmark_dataset = create_datasets(
    device = device,
    include_all_datasets = True
)

train_dataloader, val_dataloader, benchmark_dataloader = create_dataloaders(
    train_dataset = train_dataset,
    val_dataset = val_dataset,
    benchmark_dataset = benchmark_dataset,
    batch_size = batch_size,
    val_size = val_size
)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [ ]:
model, train_loss_hist, val_performance_hist, best_val_performance = training_loop(
    model = model,
    criterion = criterion,
    optimizer = optimizer,
    train_dataloader = train_dataloader,
    val_dataloader = val_dataloader,
    dbx_access_token = dbx_access_token,
    num_epochs = num_epochs,
    critiqueing_metric = "Mean Pixel Accuracy",
    auto_stop = False
)

In [ ]:
graph_loss_history(train_loss_hist)

graph_performance_history(
    performance_hist = val_performance_hist,
    split = 'Val',
    metrics = ["Mean Pixel Accuracy", "Precision"]
)

show_sample_results(
    model = model,
    dataset = train_dataset,
    device = device,
)

In [ ]:
test_model_on_benchmarks(
    model = model,
    device = device,
    all_benchmarks = True,
    report_results = True
)

Push Changes (when ready)

In [ ]:
# Fill in user info
git_username = ""
git_email = ""

while not git_username:
    commit_message = input("Username left empty.\nGitHub username: ")
while not git_email:
    commit_message = input("Email left empty.\nGitHub email: ")

!git config --global user.name $git_username
!git config --global user.email $git_email

git_access_token = getpass("Enter your GitHub access token: ")

git_push_url = f"https://{git_username}:{git_access_token}@{git_repo_url.replace('https://','')}"

commit_message = input("Commit message: ")
while not commit_message:
    commit_message = input("Commit message cannot be empty\nCommit message: ")

!git add .
!git commit -m "{commit_message}"
!git push $git_push_url $git_branch